### PyTorch Lightning

In [ ]:
import sys
sys.path.append("..")

import pytorch_lightning as pl
import torch
# from models.TwoLayerMLP import MLPModel
from models.MlpClassifierModel import MlpClassifierModel

In [ ]:
print(f"Lightning version: {pl.__version__}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
hparams = {}
hparams['hidden_size'] = 1024
hparams['lr'] = 2e-3
hparams['lr_decay_rate'] = 0.25
hparams['batch_size'] = 256
# hparams['activation'] = 'GELU'
hparams['activation'] = 'ReLU'
# model = MLPModel(hparams).to('cuda')
model = MlpClassifierModel(hparams).to('cuda')
# model = model.to(device)
model

In [ ]:
# model.prepare_data()

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=10, verbose=False, mode="min")

In [ ]:
# https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html
trainer = None

# from pytorch_lightning.callbacks import EarlyStopping

# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=10,
# )

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
early_stop_callback = EarlyStopping(
    monitor="val_loss", 
    min_delta=0.00, 
    patience=10, 
    verbose=False, 
    mode="min")

trainer = pl.Trainer(
    profiler="simple",
    max_epochs=300,
    gpus=1,
    callbacks=[early_stop_callback],
    check_val_every_n_epoch=1,
)

trainer.fit(model)

In [ ]:
model.eval()

In [ ]:
preds_new, acc, labels = model.getTestAcc()

In [ ]:
acc

In [ ]:
preds_new

In [ ]:
model.state_dict()

In [ ]:
# preds_old = preds_new

In [ ]:
preds_old

In [ ]:
# !tensorboard --logdir=lightning_logs

In [ ]:
# from pytorch_lightning.callbacks import ModelCheckpoint

# checkpoint_callback = ModelCheckpoint(dirpath="/media/ssk/DATA/GRP_code/gr_football_analytics/notebooks/lightning_logs/version_0/checkpoints")
# trainer = pl.Trainer(callbacks=[checkpoint_callback])
# hparams = {}
# hparams['hidden_size'] = 256
# hparams['lr'] = 0.00001
# hparams['lr_decay_rate'] = 0.25
# hparams['batch_size'] = 256
# # hparams['activation'] = 'LeakyReLU'
# hparams['activation'] = 'ReLU'
# model = MLPModel(hparams)
# model = model.to('cuda')
# trainer.fit(model)
# checkpoint_callback.best_model_path

#### Confusion matrix

In [ ]:
hparams = {}
hparams['hidden_size'] = 1024
hparams['lr'] = 2e-3
hparams['lr_decay_rate'] = 0.25
hparams['batch_size'] = 256
hparams['activation'] = 'GELU'
# hparams['activation'] = 'ReLU'
# model = MLPModel(hparams).to('cuda')
model = MlpClassifierModel(hparams).to('cuda')

In [ ]:
# checkpoint_path = '/home/ssk/ray_results/DEFAULT_2022-02-14_11-59-48/imitation_learning_3a12f_00000/checkpoint_000042/checkpoint'
# checkpoint_path = "/home/ssk/ray_results/DEFAULT_2022-02-14_11-59-48/imitation_learning_3a12f_00000/checkpoint_000099/checkpoint"
# checkpoint_path = "/media/ssk/DATA/GRP_code/gr_football_analytics/notebooks/lightning_logs/version_0/checkpoints/epoch=3-step=13507.ckpt"
checkpoint_path = "/media/ssk/DATA/GRP_code/gr_football_analytics/notebooks/lightning_logs/version_27/checkpoints/epoch=208-step=681548.ckpt"
# checkpoint_path = "/media/ssk/DATA/GRP_code/gr_football_analytics/notebooks/lightning_logs/version_35/checkpoints/epoch=276-step=1106891.ckpt"
checkpoint_dict = torch.load(checkpoint_path)

In [ ]:
model.load_state_dict(checkpoint_dict['state_dict']) 
model.to('cuda')
model.eval()

In [ ]:
model.prepare_data()

In [ ]:
a = {'a':1, 'b':2}
for k, v in a:
    print(k, v)

In [ ]:
action_classes = {
    0: "idle", #ignored
    1: "left",
    2: "top_left",
    3: "top",
    4: "top_right",
    5: "right", #ignored
    6: "bottom_right",
    7: "bottom",
    8: "bottom_left",
    9: "long_pass",
    10: "high_pass",
    11: "short_pass",
    12: "shot",
    13: "sprint",
    14: "release_direction", #ignored
    15: "release_sprint", #ignored
    16: "sliding",
    17: "dribble", #ignored
    18: "release_dribble", #ignored
}

In [ ]:
actions = []

for frame in model.dict_dataset:
    actions.append(action_classes[model.dict_dataset[frame][115]])
    
print(len(actions))

#### Action distribution of the training data

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

plt.figure(figsize = (10,7))
ax = sn.histplot(actions, shrink=0.5)
# ax = sn.barplot(y_pred, x=[classes[key] for key in classes.keys()])
# ax.set(xlabel='actions predicted', ylabel='count')
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.xlabel('Actions')
plt.title('Action distributions of the training data')
plt.savefig('action_class_dist_train_data')
plt.show()

In [ ]:
preds, acc, labels = model.getTestAcc()

In [ ]:
acc*100

#### Tesnorboard

In [ ]:
!tensorboard --logdir=lightning_logs

#### Calculate F1 score

In [ ]:
from sklearn.metrics import fbeta_score
y_pred, acc, y_true = model.getTestAcc()
y_pred += 1 # because no idle
from sklearn.metrics import fbeta_score
print("F1 : {}".format(fbeta_score(y_true, y_pred, average='macro', beta=0.5)))

In [ ]:
action_set_v1_used=[]
for i in np.unique(y_pred):
    action_set_v1_used.append(action_classes[i])
len(action_set_v1_used)

In [ ]:
action_set_v1_used=[]
for i in np.unique(y_true):
    action_set_v1_used.append(action_classes[i])
len(action_set_v1_used)

In [ ]:
from sklearn import metrics
res = metrics.classification_report(y_true, y_pred, digits=3, output_dict=True,target_names = action_set_v1_used)
df = pd.DataFrame(res).transpose()
df=df.drop(['accuracy', 'macro avg', 'weighted avg'])
df.sort_values("f1-score")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# y_pred = []
# y_true = []


y_pred, acc, y_true = model.getTestAcc()
# iterate over test data
# for inputs, labels in testloader:
#         output = net(inputs) # Feed Network

#         output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
#         y_pred.extend(output) # Save Prediction
        
#         labels = labels.data.cpu().numpy()
#         y_true.extend(labels) # Save Truth

# constant for classes
classes = {
#     0: "idle",
    1: "left",
    2: "top_left",
    3: "top",
    4: "top_right",
#     5: "right",
    6: "bottom_right",
    7: "bottom",
    8: "bottom_left",
    9: "long_pass",
    10: "high_pass",
    11: "short_pass",
    12: "shot",
    13: "sprint",
#     14: "release_direction",
#     15: "release_sprint",
    16: "sliding",
#     17: "dribble",
#     18: "release_dribble",
}

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
# print(np.sum(cf_matrix))
df_cm = pd.DataFrame(cf_matrix*100/np.sum(cf_matrix), index = [classes[key] for key in classes.keys()],
                     columns = [classes[key] for key in classes.keys()])
# print(df_cm.head())
plt.figure(figsize = (24,14))
# plt.plot(df_cm)
sn.heatmap(df_cm, annot=True, vmin=0, vmax=100, cmap="YlGnBu")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.jpg')

# left: true, bottom: predicted

In [ ]:
# df_cm

In [ ]:
classes = {
    0: "idle",
    1: "left",
    2: "top_left",
    3: "top",
    4: "top_right",
    5: "right",
    6: "bottom_right",
    7: "bottom",
    8: "bottom_left",
    9: "long_pass",
    10: "high_pass",
    11: "short_pass",
    12: "shot",
    13: "sprint",
    14: "release_direction",
    15: "release_sprint",
    16: "sliding",
    17: "dribble",
    18: "release_dribble",
}

### Plots of different actions

In [ ]:
y_pred, acc, y_true = model.getTestAcc()

In [ ]:
plt.figure(figsize = (10,7))
ax = sn.histplot([classes[pred] for pred in y_true], shrink=0.5)
# ax = sn.barplot(y_pred, x=[classes[key] for key in classes.keys()])
# ax.set(xlabel='actions predicted', ylabel='count')
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.show()

In [ ]:
plt.figure(figsize = (10,7))
ax = sn.histplot([classes[pred] for pred in y_pred], shrink=0.5, bins=19)
# ax = sn.barplot(y_pred, x=[classes[key] for key in classes.keys()])
# ax.set(xlabel='actions predicted', ylabel='count')
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.show()

In [ ]:
data = {"original_actions":[classes[pred] for pred in y_true], "predicted_actions":[classes[pred] for pred in y_pred]}
# data = {"original_actions": y_true, "predicted_actions": y_pred}

In [ ]:
action_data = pd.DataFrame(data)

In [ ]:
action_data

In [ ]:
action_data['predicted_actions'].isnull().values.sum()

In [ ]:
plt.figure(figsize = (20,14))
ax = sn.histplot(action_data, x="original_actions", multiple="dodge", shrink=.8)
# ax = sn.barplot(y_pred, x=[classes[key] for key in classes.keys()])
# ax.set(xlabel='actions predicted', ylabel='count')
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.show()

In [ ]:
# y_true_list = list(y_true)
# original_pred = [y_true.count(pred) for pred in list(y_true)]

In [ ]:
# action_data[['original_actions']].groupby('original_actions').count()
original_action_counts = action_data['original_actions'].value_counts()
predicted_action_counts = action_data['predicted_actions'].value_counts()
len(original_action_counts.values), len(predicted_action_counts.values)

In [ ]:
classes = {
    0: "idle",
    1: "left",
    2: "top_left",
    3: "top",
    4: "top_right",
    5: "right",
    6: "bottom_right",
    7: "bottom",
    8: "bottom_left",
    9: "long_pass",
    10: "high_pass",
    11: "short_pass",
    12: "shot",
    13: "sprint",
    14: "release_direction",
    15: "release_sprint",
    16: "sliding",
    17: "dribble",
    18: "release_dribble",
}

# for action in classes.values():
#     if action not in predicted_action_counts.keys():
#         print(action)
#         predicted_action_counts[action] = 0
        
len(predicted_action_counts.values)

In [ ]:

action_data = pd.DataFrame({"predicted_actions": predicted_action_counts.values, "original_actions": original_action_counts.values}, index=predicted_action_counts.keys())
action_data

#### Used as an example

In [ ]:
# Numbers of pairs of bars you want
N = 3

# Data on X-axis

# Specify the values of blue bars (height)
blue_bar = (23, 25, 17)
# Specify the values of orange bars (height)
orange_bar = (19, 18, 14)

# Position of bars on x-axis
ind = np.arange(N)

# Figure size
plt.figure(figsize=(10,5))

# Width of a bar 
width = 0.3       

# Plotting
plt.bar(ind, blue_bar , width, label='Blue bar label')
plt.bar(ind + width, orange_bar, width, label='Orange bar label')

plt.xlabel('Here goes x-axis label')
plt.ylabel('Here goes y-axis label')
plt.title('Here goes title of the plot')

# xticks()
# First argument - A list of positions at which ticks should be placed
# Second argument -  A list of labels to place at the given locations
plt.xticks(ind + width / 2, ('Xtick1', 'Xtick3', 'Xtick3'))

# Finding the best position for legends and putting it
plt.legend(loc='best')
plt.show()

In [ ]:
a = np.array([x for x in range(19)])
a+5

In [ ]:
plt.figure(figsize=(20,10))
width = 0.3  
ind = np.array([x for x in range(13)])
plt.bar(ind, action_data['original_actions'], width, label="original action", color="steelblue")
plt.bar(ind+width, action_data['predicted_actions'], width, label="predicted action", color="salmon")

locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.xticks(ind + width / 2, action_data.index)
plt.xlabel('actions')
plt.ylabel('counts')
# plt.title('Here goes title of the plot')
plt.legend(loc='best')

plt.savefig('original_vs_predicted_action_class_distributions.jpg')
plt.show()




# plt.bar(ind, blue_bar , width, label='Blue bar label')
# plt.bar(ind + width, orange_bar, width, label='Orange bar label')

In [ ]:
!tensorboard --logdir=lightning_logs